In [ ]:
import os
import numpy as np
import xarray as xr
import matplotlib as mpl
import cartopy.crs as ccrs
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors

from datetime import datetime

In [ ]:
def my_color_map ():

    # This function was developped by Kai-Yuan Cheng
    # source code from Kai-Yuan Cheng

    # get colormap
    ncolors = 256
    colors = plt.cm.binary_r(np.linspace(0.0,1.0,256))

    # change alpha values
    colors[:,-1] = np.linspace(0.0,1.0,ncolors)

    # create a colormap object
    mymap = mcolors.LinearSegmentedColormap.from_list(name='my_color_map',colors=colors)

    # register this new colormap with matplotlib
    if 'my_color_map' not in mpl.colormaps:
        mpl.colormaps.register(cmap=mymap)

In [ ]:
def main (icdate,fctidx,case,resolution,domain,latlon,dataroot,figdir,version):

    yyyy = icdate[0:4]
    mm = icdate[4:6]
    dd = icdate[6:8]
    hh = icdate[9:11]

    # reference: https://www.gfdl.noaa.gov/visualization/visualizations-mesoscale-dynamics/
    # reference: https://www.star.nesdis.noaa.gov/goes/index.php
    # reference: https://www.star.nesdis.noaa.gov/goes/fulldisk.php?sat=G16
    # reference: https://www.star.nesdis.noaa.gov/goes/fulldisk.php?sat=G18
    # reference: https://mtarchive.geol.iastate.edu

    fbg = f'{figdir}/world.topo.bathy.200404.3x5400x2700.png'
    #fbg = f'{figdir}/world.topo.bathy.200404.3x21600x10800.png'
    img = plt.imread(fbg)

    print(f'After Loading Image: {datetime.now()}')

    if domain == 'GLOBAL':
        lonw = '180W'
        lone = '180E'
        lats = '90S'
        latn = '90N'
        projection = ccrs.PlateCarree()
    if domain == 'GOES16':
        lonw = '165.2W'
        lone = '14.8E'
        lats = '90S'
        latn = '90N'
        projection = ccrs.Geostationary(central_longitude=-75.2,satellite_height=35785831)
    if domain == 'GOES17':
        lonw = '132.7E'
        lone = '47.3W'
        lats = '90S'
        latn = '90N'
        projection = ccrs.Geostationary(central_longitude=-137.3,satellite_height=35785831)
    if domain == 'Himawari9':
        lonw = '50.7E'
        lone = '129.3W'
        lats = '90S'
        latn = '90N'
        projection = ccrs.Geostationary(central_longitude=140.7,satellite_height=35785831)
    if domain == 'Meteosat9':
        lonw = '44.5W'
        lone = '135.5E'
        lats = '90S'
        latn = '90N'
        projection = ccrs.Geostationary(central_longitude=45.5,satellite_height=35785831)
    if domain == 'Meteosat10':
        lonw = '90W'
        lone = '90E'
        lats = '90S'
        latn = '90N'
        projection = ccrs.Geostationary(central_longitude=0.0,satellite_height=35785831)
    if domain == 'FY4B':
        lonw = '15E'
        lone = '165W'
        lats = '90S'
        latn = '90N'
        projection = ccrs.Geostationary(central_longitude=105.0,satellite_height=35785831)
    if domain == 'AREA':
        lonw = '0E'
        lone = '0E'
        lats = '0N'
        latn = '0N'
        projection = ccrs.PlateCarree()

    plt.style.use('dark_background')
    fig, ax = plt.subplots(1,figsize=(11,11),subplot_kw={'projection':projection})
    projection = ccrs.PlateCarree()
    img_extent = (-180, 180, -90, 90)

    ax.imshow(img,extent=img_extent,transform=projection)

    print(f'After Ploting Image: {datetime.now()}')

    fgd = f'{dataroot}/{icdate}.{resolution}.{case}/history/{yyyy}{mm}{dd}{hh}/lw_C3072_11520x5760.fre.nc'
    fcd = f'{dataroot}/{icdate}.{resolution}.{case}/history/{yyyy}{mm}{dd}{hh}/iw_C3072_11520x5760.fre.nc'

    if (not os.path.exists(fgd)) or (not os.path.exists(fcd)):
        exit(f'Warning: {fgd} or {fcd} does not exist!')

    dgd = xr.open_dataset(fgd)
    dcd = xr.open_dataset(fcd)

    time = dcd['time'][fctidx-1].values

    lat = dgd['grid_yt'].values
    lon = dgd['grid_xt'].values
    var = dgd['lw'][fctidx-1,:,:].values + dcd['iw'][fctidx-1,:,:].values
    var = np.where(var<1.e-15,1.e-15,var)
    var = np.log(var)

    ax.pcolormesh(lon,lat,var,cmap='my_color_map',vmin=-15,vmax=0,transform=projection)

    print(f'After Ploting Cloud: {datetime.now()}')

    if domain == 'AREA':
        lat_lon = np.array(latlon.split(',')).astype(float)
        if lat_lon[0] < 0:
            lats = f'{-lat_lon[0]}S'
        if lat_lon[0] > 0:
            lats = f'{lat_lon[0]}N'
        if lat_lon[1] < 0:
            latn = f'{-lat_lon[1]}S'
        if lat_lon[1] > 0:
            latn = f'{lat_lon[1]}N'
        if lat_lon[2] < 0:
            lonw = f'{-lat_lon[2]}W'
        if lat_lon[2] > 0:
            lonw = f'{lat_lon[2]}E'
        if lat_lon[3] < 0:
            lone = f'{-lat_lon[3]}W'
        if lat_lon[3] > 0:
            lone = f'{lat_lon[3]}E'
        if lat_lon[2] < 0:
            lat_lon[2] = lat_lon[2] + 360.0
        if lat_lon[3] < 0:
            lat_lon[3] = lat_lon[3] + 360.0

    if domain == 'AREA':
        ax.set_title(f'X-SHiELD {version} Cloud Imagery [{lats}-{latn},{lonw}-{lone}] at {time} UTC [IC: {yyyy}-{mm}-{dd} {hh}:00:00 UTC]')
        ax.set_extent([lat_lon[2],lat_lon[3],lat_lon[0],lat_lon[1]],crs=projection)
    elif domain == 'GLOBAL':
        ax.set_title(f'X-SHiELD {version} Global Cloud Imagery at {time} UTC [IC: {yyyy}-{mm}-{dd} {hh}:00:00 UTC]')
        ax.set_global()
    else:
        ax.set_title(f'X-SHiELD {version} {domain}-like Cloud Imagery at {time} UTC [IC: {yyyy}-{mm}-{dd} {hh}:00:00 UTC]')
        ax.set_global()

    ax.set(frame_on=False)

    plt.tight_layout()
    plt.savefig(f'{figdir}/DT_{case}_{resolution}_{domain}_{lats}{latn}{lonw}{lone}_{icdate}_{fctidx:>03}.png',dpi=300,bbox_inches='tight')
    plt.close('all')

    print(f'After Saving Plot  : {datetime.now()}')

In [ ]:
my_color_map ()

# data root
dataroot = '/scratch/cimes/GLOBALFV3'
# experiment
icdate = '20191020.00Z'
resolution = 'C3072'
# case name: xs24v2, L79x2_pire
case = 'xs24v2'
# model version
version = '2024'
# time step
fctidx = 6
# display area: GLOBAL, GOES16, GOES17, Himawari9, Meteosat9, Meteosat10, FY4B, AREA 
domain = 'FY4B'
# define the area when domain = 'AREA', format: lats,latn,lonw,lone
latlon = '30,45,-85,-70'
# output figure directory and background directory
figdir = 'figures'
if not os.path.exists(figdir):
    os.makedirs(figdir)

print(f'Start Time         : {datetime.now()}')
main (icdate,fctidx,case,resolution,domain,latlon,dataroot,figdir,version)
print(f'End Time           : {datetime.now()}')